In [1]:
from bins import Bins
from utils import calcAllChi2, calcOneChi2, HistMaker
from ROOT import TFile, TH1
import math

bins = Bins.readFrom("ranges.yml")
histMakerData = HistMaker("apr12_diele_088_090_ag123ag_2500A_accepted_np_2.dat", "_data", bins)
histsData = histMakerData.makeHists()
histMakerMC = HistMaker("medium_isotropic_eff_ag1230ag_np_9deg.dat", "_MC", bins    )

#outfile = TFIle("out.root","RECREATE")
allHistsMC = []

#calcAllChi2(histsMC, histsData)

Welcome to JupyROOT 6.28/04
iter 0
Before processing events 15:18:11
After processing events 15:18:11
iter 1
Before processing events 15:18:45
After processing events 15:18:47


In [2]:
import torch

torch.manual_seed(0)

import gpytorch
import botorch

import matplotlib.pyplot as plt

plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (8, 6)

from tqdm.notebook import tqdm

import warnings

In [3]:
TH1.SetDefaultSumw2

N_PARAMS = 3

In [4]:
#lb = -1
#ub = 1

#bounds = torch.tensor([[lb]*N_PARAMS, [ub]*N_PARAMS], dtype=torch.float)
bounds = torch.tensor([[0, 0,         0],
                       [1, 2*math.pi, 2]], dtype=torch.float)

grid_r   = torch.linspace(bounds[0][0], bounds[1][0], 101)
grid_phi = torch.linspace(bounds[0][1], bounds[1][1], 101)
grid_z   = torch.linspace(bounds[0][2], bounds[1][2], 101)

grid_x1, grid_x2, grid_x3 = torch.meshgrid(grid_r, grid_phi, grid_z, indexing="ij")

xs = torch.vstack([grid_x1.flatten(), grid_x2.flatten(), grid_x3.flatten()]).transpose(-2,-1)

In [5]:
class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=1)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def fit_gp_model(train_x, train_y, num_train_iters=500):
    # declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # train the hyperparameter (the constant)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in range(num_train_iters):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

In [6]:
num_queries = 200
num_repeats = 1
num_samples = 1

In [7]:
def lambdas(xx):
   # return x[0], x[1], x[2]
    r, phi, z = xx[0], xx[1], xx[2]
    x = r*math.cos(phi)
    y = r*math.sin(phi)
    lambda_theta = 0.5 * (2*x + z)
    lambda_phi   = 0.25 * (-2 - 2*x + 3*z)
    lambda_theta_phi = y/math.sqrt(2.)
    return lambda_theta, lambda_phi, lambda_theta_phi

def all_lambdas(xx):
    def generator(xx):
        for x in xx:
            lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)
            yield torch.tensor([lambda_theta, lambda_phi, lambda_theta_phi])
           # yield torch.tensor([1./(chi2 / ndf)])
    return torch.stack([a for a in generator(xx)])

def constraint1(xx):
    print("In the constraint fucntion, xx = ", xx)
    input_shape = xx.shape
    ll = list(xx.shape)
    ll[-1] = 1
    if len(input_shape)>1:
        xx = torch.reshape(xx, input_shape[1:])
        output_shape = tuple(ll)
        result = 1+xx[0,0]+xx[0,1]
    else:
        result = 1+xx[0]+xx[1]
        output_shape = (1,)
        
    print(xx)
    print(input_shape)
    print(result)
    print(output_shape)
    
    return torch.reshape(torch.tensor([result],requires_grad=xx.requires_grad),output_shape)

#def cost(xx):
#    def generator(xx):
#        for x in xx:
#            lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)#
#
#            result = 0
#
#            if pow(1-lambda_phi, 2) - pow(lambda_theta-lambda_phi, 2) < 4*pow(lambda_theta_phi, 2):
#                result = 1
#            if 1 + lambda_theta + 2*lambda_phi < 0:
#                result = 1

#            yield torch.tensor([result])
#    return torch.stack([a for a in generator(xx)])
    
HIST_INDEX = 3

def objective(xx):
    def generator(xx):
        for x in xx:
            lambda_theta, lambda_phi, lambda_theta_phi = lambdas(x)

            histsMC = histMakerMC.makeHists(lambda_theta, lambda_phi, lambda_theta_phi)
            chi2, ndf = calcOneChi2(histsMC[0][HIST_INDEX], histsData[0][HIST_INDEX])
            allHistsMC.append(histsMC[0][HIST_INDEX])
            if not chi2 or not ndf:
                return torch.tensor([0])
            yield torch.tensor([1.0/(chi2 / ndf)])
           # yield torch.tensor([1./(chi2 / ndf)])
    return torch.stack([a for a in generator(xx)])

In [8]:
# -2 is the default value when no feasible has been found
default_value = 1200
feasible_incumbents = torch.ones((num_repeats, num_queries)) * default_value

for trial in range(num_repeats):
    print("trial", trial)

    torch.manual_seed(trial)
    train_x = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(num_samples, 3)
    train_utility = objective(train_x)

    for i in tqdm(range(num_queries)):
        print("query", i)

        feasible_incumbents[trial, i] = train_utility.max()
        utility_model, utility_likelihood = fit_gp_model(
            train_x, train_utility.squeeze(-1)
        )
        best_f = train_utility.max()
                
        policy = botorch.acquisition.monte_carlo.qExpectedImprovement(
            model=utility_model,
            best_f=train_utility.max(),
        )

        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=RuntimeWarning)
            next_x, acq_val = botorch.optim.optimize_acqf(
                policy,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            #    nonlinear_inequality_constraints=[constraint1],
                inequality_constraints=[(torch.tensor([0,2]),torch.tensor([-2.0,-1.0],dtype=torch.float),-2.0)],
            #    ic_generator=botorch.optim.initializers.gen_batch_initial_conditions
            )

        next_utility = objective(next_x)

        train_x = torch.cat([train_x, next_x])
        train_utility = torch.cat([train_utility, next_utility])


trial 0


  0%|          | 0/200 [00:00<?, ?it/s]

query 0


/Users/harabasz/miniforge3/envs/pyroot/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


query 1


...

/Users/harabasz/miniforge3/envs/pyroot/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


Before reweighting hists 15:34:47
After reweighting hists 15:34:47
query 198


/Users/harabasz/miniforge3/envs/pyroot/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


query 199


/Users/harabasz/miniforge3/envs/pyroot/lib/python3.11/site-packages/botorch/optim/initializers.py:403: BadInitialCandidatesWarning: Unable to find non-zero acquisition function values - initial conditions are being selected randomly.
  warnings.warn(


In [9]:
torch.save(feasible_incumbents, f"./incumbents.pth")
fout = TFile("out.root", "RECREATE")
fout.cd()
for hist in allHistsMC:
    print ("Writing hist: ", hist.GetName())
    hist.Write()
for j, hists in enumerate(histsData):
    for k, hist in enumerate(hists):
            hist.Write()
            print ("Writing hist: ", j, k, hist)
fout.Close()

Writing hist:  hist_150to300_m06m02_MC_iter2
Writing hist:  hist_150to300_m06m02_MC_iter3
Writing hist:  hist_150to300_m06m02_MC_iter4
Writing hist:  hist_150to300_m06m02_MC_iter5
Writing hist:  hist_150to300_m06m02_MC_iter6
Writing hist:  hist_150to300_m06m02_MC_iter7
Writing hist:  hist_150to300_m06m02_MC_iter8
Writing hist:  hist_150to300_m06m02_MC_iter9
Writing hist:  hist_150to300_m06m02_MC_iter10
Writing hist:  hist_150to300_m06m02_MC_iter11
Writing hist:  hist_150to300_m06m02_MC_iter12
Writing hist:  hist_150to300_m06m02_MC_iter13
Writing hist:  hist_150to300_m06m02_MC_iter14
Writing hist:  hist_150to300_m06m02_MC_iter15
Writing hist:  hist_150to300_m06m02_MC_iter16
Writing hist:  hist_150to300_m06m02_MC_iter17
Writing hist:  hist_150to300_m06m02_MC_iter18
Writing hist:  hist_150to300_m06m02_MC_iter19
Writing hist:  hist_150to300_m06m02_MC_iter20
Writing hist:  hist_150to300_m06m02_MC_iter21
Writing hist:  hist_150to300_m06m02_MC_iter22
Writing hist:  hist_150to300_m06m02_MC_ite

In [10]:
t = torch.tensor( [ [-1.0000, -0.4678,  0.5433]])
print(t)
print(t.shape)

tensor([[-1.0000, -0.4678,  0.5433]])
torch.Size([1, 3])


In [11]:
with torch.no_grad():
    predictive_distribution = utility_likelihood(utility_model(xs))
    predictive_mean = predictive_distribution.mean
    predictive_lower, predictive_upper = predictive_distribution.confidence_region()


In [12]:
from ipywidgets import interact, Layout, IntSlider
import numpy as np

axis_titles = [r"R of cone coordinates", r"$\Phi$ of cone coordinates", r"Z of cone coordinates"]

def oneplot(ax, tensor, index, cmap, title, transpose=None):
        
        global xs
        tensor_3d = torch.reshape(tensor, (101,101,101))
        xs_3d = torch.reshape(xs, (101,101,101,3))
        if transpose:
               tensor_3d = tensor_3d.transpose(transpose[0],transpose[1])
               xs_3d     = xs_3d    .transpose(transpose[0],transpose[1])
        if transpose is None:
                extent=[
                        xs_3d[0][0][0][1],
                        xs_3d[0][-1][0][1],
                        xs_3d[0][0][0][2],
                        xs_3d[0][0][-1][2],
                ]
        elif transpose == (0,1):
                extent=[
                        xs_3d[0][0][0][0],
                        xs_3d[0][-1][0][0],
                        xs_3d[0][0][0][2],
                        xs_3d[0][0][-1][2],
                ]
        elif transpose == (0,2):
                extent=[
                        xs_3d[0][0][0][1],
                        xs_3d[0][-1][0][1],
                        xs_3d[0][0][0][0],
                        xs_3d[0][0][-1][0],
                ]
        if transpose is None:
                x_title=axis_titles[1]
                y_title=axis_titles[2]
        elif transpose == (0, 1):
                x_title=axis_titles[0]
                y_title=axis_titles[2]
        elif transpose == (0,2):
                x_title=axis_titles[1]
                y_title=axis_titles[0]
        
        pos = ax.imshow(tensor_3d[index], cmap=cmap, interpolation="nearest", origin="lower", 
                vmin=tensor.min(), vmax=tensor.max(), extent=extent)
        ax.set_aspect((extent[1]-extent[0])/(extent[3]-extent[2]))
        ax.set_title(title)
        ax.set_xlabel(x_title)
        ax.set_ylabel(y_title)
        plt.colorbar(pos,fraction=0.046, pad=0.04)

def ff(x, transpose):
        cmap = "gist_rainbow"
        fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True)
        oneplot(ax[0], predictive_mean, x, cmap, "value", transpose)
        oneplot(ax[1], predictive_upper-predictive_lower, x, cmap, "confidence int. width", transpose)

def f(x):
        ff(x, None)

interact(f, x=IntSlider(50, 0, 100, 1, layout=Layout(width='500px')))


interactive(children=(IntSlider(value=50, description='x', layout=Layout(width='500px')), Output()), _dom_clas…

<function __main__.f(x)>

In [13]:
def g(x):
        ff(x, (0,1))
           
interact(g, x=IntSlider(50, 0, 100, 1, layout=Layout(width='500px')))

interactive(children=(IntSlider(value=50, description='x', layout=Layout(width='500px')), Output()), _dom_clas…

<function __main__.g(x)>

In [14]:
def h(x):
        ff(x, (0,2))
           
interact(h, x=IntSlider(50, 0, 100, 1, layout=Layout(width='500px')))

interactive(children=(IntSlider(value=50, description='x', layout=Layout(width='500px')), Output()), _dom_clas…

<function __main__.h(x)>

In [15]:
lds = all_lambdas(train_x)
lambda_thetas     = lds.transpose(-2,-1)[0]
lambda_phis       = lds.transpose(-2,-1)[1]
lambda_theta_phis = lds.transpose(-2,-1)[2]
c = torch.stack((
    torch.arange(0,200, dtype=int),
    lambda_thetas[0:200],
    lambda_phis[0:200],
    lambda_theta_phis[0:200],
    train_utility.squeeze()[0:200],
    (feasible_incumbents==feasible_incumbents.max())[0],
    feasible_incumbents[0]
),0).transpose(-2,-1)
torch.set_printoptions(precision=4,threshold=10_000, linewidth=120)
print(c)

tensor([[ 0.0000e+00,  1.4517e-01, -3.9563e-01, -3.4861e-01,  1.5016e+03,  0.0000e+00,  1.5016e+03],
        [ 1.0000e+00,  4.3991e-01, -4.5304e-03, -3.2986e-01,  2.3159e+03,  0.0000e+00,  2.3159e+03],
        [ 2.0000e+00, -3.7373e-01, -1.9562e-01, -3.7715e-01,  1.6148e+03,  0.0000e+00,  2.3159e+03],
        [ 3.0000e+00,  2.9956e-01,  8.5219e-02, -9.7784e-02,  2.6021e+03,  0.0000e+00,  2.6021e+03],
        [ 4.0000e+00,  4.2767e-01,  2.1275e-01, -9.8776e-02,  2.4187e+03,  0.0000e+00,  2.6021e+03],
        [ 5.0000e+00,  2.4926e-01,  1.1828e-01, -2.8194e-01,  2.2922e+03,  0.0000e+00,  2.6021e+03],
        [ 6.0000e+00,  2.7761e-01, -5.2928e-01,  1.5710e-02,  1.3425e+03,  0.0000e+00,  2.6021e+03],
        [ 7.0000e+00,  3.3824e-01,  2.9721e-02, -1.5726e-02,  2.7111e+03,  0.0000e+00,  2.7111e+03],
        [ 8.0000e+00,  3.7124e-01, -5.9183e-01,  5.7932e-01,  1.0154e+03,  0.0000e+00,  2.7111e+03],
        [ 9.0000e+00,  3.1788e-01, -3.6938e-01, -1.7612e-01,  1.7448e+03,  0.0000e+00,  2.7

In [28]:
bestIndex = int(c[c[:,5]>0][0][0])